In [24]:
import os
import sys
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="This sequence already has </s>.")

# Append path for module imports
scripts_path = os.path.abspath(os.path.join('..', 'scripts'))
sys.path.append(scripts_path)


# Standard library imports
import random
import string

# Third-party imports
import json
import numpy as np
import pandas as pd
import itertools
import pke
import torch
import nltk
from dateutil.parser import parse
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords, wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# from textdistance import levenshtein
from rapidfuzz import fuzz
from rapidfuzz.distance import Levenshtein as levenshtein

# Download necessary NLTK data
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')

from typing import List, Dict
import re


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\miiciii\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Import custom models from model_manager
from model_manager import (
    get_answergeneration_model,
    get_questiongeneration_model,
    get_sense2vec_model,
    get_sentence_transformer_model,
    get_random_passage
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
d:\01SetupFiles\AnacondaNavigator\anacondanavigatorfiles\envs\Thesis\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
d:\01SetupFiles\AnacondaNavigator\anacondanavigatorfiles\envs\Thesis\Lib\site-packages\sentence_trans

In [3]:
# Initialize models
t5ag_model, t5ag_tokenizer = get_answergeneration_model()
t5qg_model, t5qg_tokenizer = get_questiongeneration_model()
s2v = get_sense2vec_model()
sentence_transformer_model = get_sentence_transformer_model()
random_passage = get_random_passage()

In [37]:
def answer_question(question, context):
    """Generate an answer for a given question and context."""
    input_text = f"question: {question} context: {context}"
    input_ids = t5ag_tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        output = t5ag_model.generate(input_ids, max_length=512, num_return_sequences=1, max_new_tokens=200)

    return t5ag_tokenizer.decode(output[0], skip_special_tokens=True).capitalize()

def get_passage(passage):
    """Generate a random context from the dataset."""
    return passage.sample(n=1)['context'].values[0]

def get_question(context, answer, model, tokenizer):
    """Generate a question for the given answer and context."""
    answer_span = context.replace(answer, f"<hl>{answer}<hl>", 1) + "</s>"
    inputs = tokenizer(answer_span, return_tensors="pt")
    question = model.generate(input_ids=inputs.input_ids, max_length=50)[0]

    return tokenizer.decode(question, skip_special_tokens=True)


def get_keywords(passage):
    """Extract keywords using TF-IDF."""
    try:
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform([passage])
        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.toarray().flatten() # type: ignore
        word_scores = dict(zip(feature_names, tfidf_scores))
        sorted_words = sorted(word_scores.items(), key=lambda x: x[1], reverse=True)
        keywords = [word for word, score in sorted_words]
        return keywords
    except Exception as e:
        print(f"Error extracting keywords: {e}")
        return []

def classify_question_type(question: str) -> str:
    """
    Classify the type of question as literal, evaluative, or inferential.
    
    Parameters:
        question (str): The question to classify.
        
    Returns:
        str: The type of the question ('literal', 'evaluative', or 'inferential').
    """
    # Define keywords or patterns for each question type
    literal_keywords = [
    'what', 'when', 'where', 'who', 'how many', 'how much', 
    'which', 'name', 'list', 'identify', 'define', 'describe', 
    'state', 'mention'
    ]

    evaluative_keywords = [
    'evaluate', 'justify', 'explain why', 'assess', 'critique', 
    'discuss', 'judge', 'opinion', 'argue', 'agree or disagree', 
    'defend', 'support your answer', 'weigh the pros and cons', 
    'compare', 'contrast'
    ]

    inferential_keywords = [
    'why', 'how', 'what if', 'predict', 'suggest', 'imply', 
    'conclude', 'infer', 'reason', 'what might', 'what could', 
    'what would happen if', 'speculate', 'deduce', 'interpret', 
    'hypothesize', 'assume'
    ]


    question_lower = question.lower()
    
    # Check for literal question keywords
    if any(keyword in question_lower for keyword in literal_keywords):
        return 'literal'
    
    # Check for evaluative question keywords
    if any(keyword in question_lower for keyword in evaluative_keywords):
        return 'evaluative'
    
    # Check for inferential question keywords
    if any(keyword in question_lower for keyword in inferential_keywords):
        return 'inferential'
    
    # Default to 'unknown' if no pattern matches
    return 'unknown'

def filter_same_sense_words(original, wordlist):
    """Filter words that have the same sense as the original word."""
    try:
        base_sense = original.split('|')[1]  # Ensure there is a sense part
    except IndexError:
        print(f"Warning: The original phrase '{original}' does not have a sense part.")
        return wordlist  # Return all words if the sense part is missing

    return [word[0].split('|')[0].replace("_", " ").title().strip() for word in wordlist if word[0].split('|')[1] == base_sense]

def extract_similar_keywords(input_phrases, topn=5):
    """Call get_distractors and extract only the similar_keywords values."""
    distractors_result = get_distractors(input_phrases, topn)
    similar_keywords_list = [result["similar_keywords"] for result in distractors_result]
    return similar_keywords_list 

def get_max_similarity_score(wordlist, word):
    """Get the maximum similarity score between the word and a list of words."""
    return max(levenshtein.normalized_similarity(word.lower(), each.lower()) for each in wordlist)

def mmr(doc_embedding, word_embeddings, words, top_n, lambda_param):
    """Maximal Marginal Relevance (MMR) for keyword extraction."""
    try:
        word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
        word_similarity = cosine_similarity(word_embeddings)

        keywords_idx = [np.argmax(word_doc_similarity)]
        candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

        for _ in range(top_n - 1):
            candidate_similarities = word_doc_similarity[candidates_idx, :]
            target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

            mmr = (lambda_param * candidate_similarities) - ((1 - lambda_param) * target_similarities.reshape(-1, 1))
            mmr_idx = candidates_idx[np.argmax(mmr)]

            keywords_idx.append(mmr_idx)
            candidates_idx.remove(mmr_idx)

        return [words[idx] for idx in keywords_idx]
    except Exception as e:
        print(f"Error in MMR: {e}")
        return []

def format_phrase(phrase):
    """Format phrases by replacing spaces with underscores and adding default |n."""
    return phrase.replace(" ", "_") + "|n"


def is_valid_distractor(distractor, input_phrase):
    """Check if the distractor is valid by ensuring it's alphabetic and relevant."""
    if not re.match(r'^[a-zA-Z\s]+$', distractor):
        return False
    
    word_count = len(distractor.split())
    if word_count < 1 or word_count > 4:
        return False
    
    return True

def filter_distractors(input_phrase, similar_keywords, topn):
    """Filter distractors to ensure they match word count, aren't identical to the input, 
    and aren't too similar to each other or the input (e.g., stem similarity)."""
    word_count = len(input_phrase.split())
    filtered_keywords = []
    stemmer = PorterStemmer()
    input_stem = stemmer.stem(input_phrase.lower())

    for keyword in similar_keywords:
        keyword_stem = stemmer.stem(keyword.lower())

        if (len(keyword.split()) == word_count and 
            keyword.lower() != input_phrase.lower() and
            keyword_stem != input_stem and
            is_valid_distractor(keyword, input_phrase)):
            
            if all(stemmer.stem(kw.lower()) != keyword_stem for kw in filtered_keywords):
                filtered_keywords.append(keyword)

        if len(filtered_keywords) == topn:
            break

    return filtered_keywords


def get_distractors(input_phrases, topn=5):
    """Find similar keywords for a list of input phrases using Sense2Vec and WordNet."""
    result_list = []

    for phrase in input_phrases:
        formatted_phrase = format_phrase(phrase)

        # Check if the phrase exists in the Sense2Vec model
        if formatted_phrase in s2v:
            # Get similar phrases from Sense2Vec
            similar_phrases = s2v.most_similar(formatted_phrase, n=topn * 2)  # Get more to filter later
            similar_keywords = [item[0].split("|")[0].replace("_", " ") for item in similar_phrases]
        else:
            # List similar keys that might exist in the model for exploration
            print(f"'{formatted_phrase}' not found in the model. Exploring similar available keys...")
            available_keys = [key for key in s2v.keys() if phrase.split()[0] in key or phrase.split()[-1] in key]
            print(f"Available keys related to '{phrase}': {available_keys}")

            # Use WordNet to find synonyms if available keys are empty
            if not available_keys:
                print(f"No close match in the model for '{phrase}'. Trying WordNet for synonyms...")
                synonyms = set()
                for syn in wn.synsets(phrase.replace(" ", "_")):
                    for lemma in syn.lemmas():
                        synonyms.add(lemma.name().replace("_", " "))
                similar_keywords = list(synonyms)[:topn * 2] if synonyms else ["No match found"]
            else:
                # Provide available keys as similar suggestions
                similar_keywords = [key.split("|")[0].replace("_", " ") for key in available_keys[:topn * 2]]

        # Filter distractors to match word count, avoid identical or stem-similar words, and check format
        final_distractors = filter_distractors(phrase, similar_keywords, topn)
        # Further filter out words with the same sense
        final_distractors = filter_same_sense_words(phrase, final_distractors)
        
        result_list.append({
            "phrase": phrase,
            "similar_keywords": final_distractors
        })

    return result_list


In [38]:
def get_mca_questions(context, qg_model, qg_tokenizer, sentence_transformer_model, num_questions=5, max_attempts=2) -> List[Dict]:
    """
    Generate multiple-choice questions for a given context.
    
    Parameters:
        context (str): The context from which questions are generated.
        qg_model (T5ForConditionalGeneration): The question generation model.
        qg_tokenizer (T5Tokenizer): The tokenizer for the question generation model.
        s2v (Sense2Vec): The Sense2Vec model for finding similar words.
        sentence_transformer_model (SentenceTransformer): The sentence transformer model for embeddings.
        num_questions (int): The number of questions to generate.
        max_attempts (int): The maximum number of attempts to generate questions.
    
    Returns:
        list: A list of dictionaries with questions and their corresponding distractors.
    """
    output_list = []

    imp_keywords = get_keywords(context)
    print(f"[DEBUG] Length: {len(imp_keywords)}, Extracted keywords: {imp_keywords}")

    generated_questions = set()
    generated_answers = set()
    attempts = 0

    while len(output_list) < num_questions and attempts < max_attempts:
        attempts += 1

        for keyword in imp_keywords:
            if len(output_list) >= num_questions:
                break
            
            question = get_question(context, keyword, qg_model, qg_tokenizer)
            print(f"[DEBUG] Generated question: '{question}' for keyword: '{keyword}'")
            
            # Encode the new question
            new_question_embedding = sentence_transformer_model.encode(question, convert_to_tensor=True)
            is_similar = False

            # Check similarity with existing questions
            for generated_q in generated_questions:
                existing_question_embedding = sentence_transformer_model.encode(generated_q, convert_to_tensor=True)
                similarity = cosine_similarity(new_question_embedding.unsqueeze(0), existing_question_embedding.unsqueeze(0))[0][0]

                if similarity > 0.8:
                    is_similar = True
                    print(f"[DEBUG] Question '{question}' is too similar to an existing question, skipping.")
                    break

            if is_similar:
                continue

            # Generate and check answer
            t5_answer = answer_question(question, context)
            print(f"[DEBUG] Generated answer: '{t5_answer}' for question: '{question}'")
            
            # Skip answers longer than 3 words
            if len(t5_answer.split()) > 3:
                print(f"[DEBUG] Answer '{t5_answer}' is too long, skipping.")
                continue

            if t5_answer in generated_answers:
                print(f"[DEBUG] Answer '{t5_answer}' has already been generated, skipping question.")
                continue

            generated_questions.add(question)
            generated_answers.add(t5_answer)

            # Generate distractors
            distractors = extract_similar_keywords([t5_answer], topn=5)[0]
            print(f"list of distractors : {distractors}")
            print(f"length of distractors {len(distractors)}")
            print(f"type : {type(distractors)}")

            # Remove any distractor that is the same as the correct answer
            distractors = [d for d in distractors if d.lower() != t5_answer.lower()]
            print(f"Filtered distractors (without answer): {distractors}")

            # Ensure there are exactly 3 distractors
            if len(distractors) < 3:
                # Fill with random keywords from the imp_keywords list until we have 3 distractors
                while len(distractors) < 3:
                    random_keyword = random.choice(imp_keywords)
                    # Ensure the random keyword isn't the same as the answer or already a distractor
                    if random_keyword.lower() != t5_answer.lower() and random_keyword not in distractors:
                        distractors.append(random_keyword)

            # Limit to 3 distractors
            distractors = distractors[:3]

            print(f"[DEBUG] Final distractors: {distractors} for question: '{question}'")

            choices = distractors + [t5_answer]
            choices = [item.title() for item in choices]
            random.shuffle(choices)
            print(f"[DEBUG] Options: {choices} for answer: '{t5_answer}'")

            # Classify question type
            question_type = classify_question_type(question)
            
            output_list.append({
                'answer': t5_answer,
                'answer_length': len(t5_answer),
                'choices': choices,
                'passage': context,
                'passage_length': len(context),
                'question': question,
                'question_length': len(question),
                'question_type': question_type
            })

        print(f"[DEBUG] Generated {len(output_list)} questions so far after {attempts} attempts")

    return output_list[:num_questions]

In [40]:
original_context = get_passage(random_passage)

print(original_context)

questions_and_distractors = get_mca_questions(original_context, t5qg_model, t5qg_tokenizer, sentence_transformer_model, num_questions=5)

OUTPUT_FILE = os.path.join('..', 'outputs', 'plots', 'predictions', 'generated_questions.json')

def convert_to_serializable(data):
    if isinstance(data, pd.Series):
        return data.tolist()
    elif isinstance(data, dict):
        return {k: convert_to_serializable(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_to_serializable(i) for i in data]
    else:
        return data

try:
    with open(OUTPUT_FILE, 'r') as f:
        existing_data = json.load(f)
        if not isinstance(existing_data, list):  # Ensure existing data is a list
            raise ValueError("The existing data is not a list.")
except FileNotFoundError:
    existing_data = []
except json.JSONDecodeError:
    print(f"Warning: {OUTPUT_FILE} contains invalid JSON. Initializing with an empty list.")
    existing_data = []
except ValueError as e:
    print(f"ValueError: {e}")
    existing_data = []
except Exception as e:
    print(f"An unexpected error occurred while reading {OUTPUT_FILE}: {e}")
    existing_data = []

# Convert any non-serializable data to a serializable format
questions_and_distractors_serializable = convert_to_serializable(questions_and_distractors)
existing_data.extend(questions_and_distractors_serializable)

# Write updated data back to the file
try:
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(existing_data, f, indent=4)
    print(f"Data successfully written to {OUTPUT_FILE}.")
except Exception as e:
    print(f"An error occurred while writing to {OUTPUT_FILE}: {e}")

New Delhi is a major junction in the Indian railway network and is the headquarters of the Northern Railway. The five main railway stations are New Delhi railway station, Old Delhi, Nizamuddin Railway Station, Anand Vihar Railway Terminal and Sarai Rohilla. The Delhi Metro, a mass rapid transit system built and operated by Delhi Metro Rail Corporation (DMRC), serves many parts of Delhi and the neighbouring cities Faridabad, Gurgaon, Noida and Ghaziabad. As of August 2011, the metro consists of six operational lines with a total length of 189 km (117 mi) and 146 stations, and several other lines are under construction. It carries millions of passengers every day. In addition to the Delhi Metro, a suburban railway, the Delhi Suburban Railway exists.
[DEBUG] Length: 56, Extracted keywords: ['delhi', 'railway', 'metro', 'lines', 'new', 'station', 'stations', 'suburban', '117', '146', '189', '2011', 'addition', 'anand', 'august', 'built', 'carries', 'cities', 'consists', 'construction', 'co

Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated question: 'What is the name of the Delhi Suburban Railway?' for keyword: 'delhi'
[DEBUG] Generated answer: 'Delhi suburban railway' for question: 'What is the name of the Delhi Suburban Railway?'
'Delhi_suburban_railway|n' not found in the model. Exploring similar available keys...
Available keys related to 'Delhi suburban railway': ['New_Delhi|GPE', '^railway|NOUN', 'Delhi_Police|ORG', 'Delhi|NOUN', 'railways|NOUN', 'railway_line|NOUN', 'Delhi|ORG', 'railway|NOUN', 'Delhi_govt|NOUN', 'Delhi_police|NOUN', 'railway_tracks|NOUN', 'railway_track|NOUN', 'railway_lines|NOUN', 'railway_bridge|NOUN', 'railway_system|NOUN', 'Delhi|GPE', 'railway_stations|NOUN', 'Delhi_elections|NOUN', 'railway_station|NOUN', 'Delhi|PERSON', 'Delhi_BJP|NOUN', 'Delhi_government|NOUN']
list of distractors : []
length of distractors 0
type : <class 'list'>
Filtered distractors (without answer): []
[DEBUG] Final distractors: ['gurgaon', 'cities', 'august'] for question: 'What is the name of the De

Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated question: 'New Delhi is a major junction in the Indian what network?' for keyword: 'railway'
[DEBUG] Generated answer: 'Railway' for question: 'New Delhi is a major junction in the Indian what network?'
'Railway|n' not found in the model. Exploring similar available keys...
Available keys related to 'Railway': ['Indian_Railways|ORG', 'Railway|NOUN', 'Railways|NOUN']
list of distractors : []
length of distractors 0
type : <class 'list'>
Filtered distractors (without answer): []
[DEBUG] Final distractors: ['headquarters', 'serves', 'exists'] for question: 'New Delhi is a major junction in the Indian what network?'
[DEBUG] Options: ['Serves', 'Railway', 'Headquarters', 'Exists'] for answer: 'Railway'


...

[DEBUG] Generated answer: 'New delhi railway station, old delhi, nizamuddin railway station, anand vihar railway terminal and sarai rohilla' for question: 'What is the name of the five main railways in New Delhi?'
[DEBUG] Answer 'New delhi railway station, old delhi, nizamuddin railway station, anand vihar railway terminal and sarai rohilla' is too long, skipping.
[DEBUG] Generated question: 'What are the five main railways in New Delhi?' for keyword: 'stations'


Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: 'New delhi railway station, old delhi, nizamuddin railway station, anand vihar railway terminal and sarai rohilla' for question: 'What are the five main railways in New Delhi?'
[DEBUG] Answer 'New delhi railway station, old delhi, nizamuddin railway station, anand vihar railway terminal and sarai rohilla' is too long, skipping.
[DEBUG] Generated question: 'What type of railway is the Delhi Metro?' for keyword: 'suburban'
[DEBUG] Question 'What type of railway is the Delhi Metro?' is too similar to an existing question, skipping.
[DEBUG] Generated question: 'How many miles long is the Delhi Metro?' for keyword: '117'
[DEBUG] Question 'How many miles long is the Delhi Metro?' is too similar to an existing question, skipping.
[DEBUG] Generated question: 'How many stations are in the Delhi Metro?' for keyword: '146'
[DEBUG] Question 'How many stations are in the Delhi Metro?' is too similar to an existing question, skipping.
[DEBUG] Generated question: 'How many k

Both `max_new_tokens` (=200) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[DEBUG] Generated answer: '2011' for question: 'In what year did the Delhi Metro begin operating?'
'2011|n' not found in the model. Exploring similar available keys...
Available keys related to '2011': ['the_summer_of_2011|DATE', 'between_2011_and_2013|DATE', '2010-2011|DATE', 'October_2011|DATE', 'http://www.theguardian.com/commentisfree/2011/jun/09/iran|X', '2011|CARDINAL', 'February_2011|DATE', 'March_2011|DATE', 'early_2011|DATE', '2011-2014|DATE', 'between_2007_and_2011|DATE', '2011/12|NUM', 'June_2011|DATE', '2011_to_2013|DATE', 'November_2011|DATE', '^2011|PUNCT', 'MITB_2011|DATE', 'November_of_2011|DATE', 'around_2011|DATE', '2005-2011|DATE', 'Summer_2011|DATE', '2010/2011|CARDINAL', 'LGA_2011|DATE', '2011-3|DATE', 'December_of_2011|DATE', 'content/uploads/2011/09/mind_blown.gif|NOUN', 'cpu-cooler-nhd14se2011|NOUN', 'http://www.torontosun.com/2011/11/01/strong|X', '2011-2012|DATE', '2011|NUM', 'April_2011|DATE', 'July_2011|DATE', 'http://www.theguardian.com/technology/2011/mar/